# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** While choosing the parameters I look at the papers provided in the links as well as used my own intuition. I chose the batch_size and vocab_threshold to be rather small, so that the training process is steady and stable. The size of embedding around 256 seems to be big enough, so that the training process does not take too long and information is not lost at the same time. In addition, 512 features in the hidden state seems to be large enough for an optimal performance, since our goal was not perfection. 


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** I chose to keep the transform_train method as it is, because it was using the most popular preprocessing techniques, therefore, it is a reliable choice. Feature augmentation (such as RandomVerticalFlipping or RandomHorizontalCrop) would not be very relevant here, because standing person is something different than a person standing upside down. The image size of 224x224 is most relevant for resnet pre-trained weights.


### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** I selected to train the decoder weights and only the embedding layer of the encoder. The reason for this is that pre-trained CNN Resnet already has a lot of useful weights. Therefore, we can expect it to give reasonable results while we can also save some training size by reducing the number of parameters.

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** I chose Adamax optimizer, because it is considered a superior version of Adam algorithm, especially in the cases when model has embeddings. (https://arxiv.org/abs/1412.6980)

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 10          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = False    # if True, load existing vocab file
embed_size = 256           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)


loading annotations into memory...
Done (t=0.90s)
creating index...
index created!
[0/414113] Tokenizing captions...
[100000/414113] Tokenizing captions...
[200000/414113] Tokenizing captions...
[300000/414113] Tokenizing captions...
[400000/414113] Tokenizing captions...
loading annotations into memory...
Done (t=0.93s)
creating index...


  0%|          | 887/414113 [00:00<01:32, 4453.09it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [01:36<00:00, 4286.52it/s]


In [7]:
import importlib
import model
importlib.reload(model)
from model import EncoderCNN, DecoderRNN

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)

decoder = DecoderRNN(embed_size, hidden_size, vocab_size, batch_size)

/opt/conda/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [8]:

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adamax(params)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [9]:
import torch.utils.data as data
import numpy as np
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
        
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [100/41412], Loss: 4.5214, Perplexity: 91.9626
Epoch [1/3], Step [200/41412], Loss: 4.0597, Perplexity: 57.95882
Epoch [1/3], Step [300/41412], Loss: 4.2015, Perplexity: 66.78486
Epoch [1/3], Step [400/41412], Loss: 3.2306, Perplexity: 25.29395
Epoch [1/3], Step [500/41412], Loss: 4.0374, Perplexity: 56.68066
Epoch [1/3], Step [600/41412], Loss: 3.2464, Perplexity: 25.69825
Epoch [1/3], Step [700/41412], Loss: 3.5822, Perplexity: 35.9524
Epoch [1/3], Step [800/41412], Loss: 4.0962, Perplexity: 60.10892
Epoch [1/3], Step [900/41412], Loss: 3.1970, Perplexity: 24.45947
Epoch [1/3], Step [1000/41412], Loss: 3.4185, Perplexity: 30.5224
Epoch [1/3], Step [1100/41412], Loss: 3.4976, Perplexity: 33.03461
Epoch [1/3], Step [1200/41412], Loss: 3.6770, Perplexity: 39.5277
Epoch [1/3], Step [1300/41412], Loss: 3.4396, Perplexity: 31.1737
Epoch [1/3], Step [1400/41412], Loss: 2.8990, Perplexity: 18.1562
Epoch [1/3], Step [1500/41412], Loss: 3.5555, Perplexity: 35.0066
Epoch [1/3]

Epoch [1/3], Step [24600/41412], Loss: 2.5717, Perplexity: 13.0885
Epoch [1/3], Step [24700/41412], Loss: 2.2404, Perplexity: 9.39684
Epoch [1/3], Step [24800/41412], Loss: 2.7673, Perplexity: 15.9153
Epoch [1/3], Step [24900/41412], Loss: 3.1234, Perplexity: 22.7230
Epoch [1/3], Step [25000/41412], Loss: 1.9501, Perplexity: 7.02916
Epoch [1/3], Step [25100/41412], Loss: 2.7139, Perplexity: 15.0879
Epoch [1/3], Step [25200/41412], Loss: 2.6800, Perplexity: 14.5847
Epoch [1/3], Step [25300/41412], Loss: 2.4455, Perplexity: 11.5362
Epoch [1/3], Step [25400/41412], Loss: 2.2269, Perplexity: 9.27084
Epoch [1/3], Step [25500/41412], Loss: 2.5771, Perplexity: 13.15841
Epoch [1/3], Step [25600/41412], Loss: 2.2757, Perplexity: 9.73500
Epoch [1/3], Step [25700/41412], Loss: 2.3480, Perplexity: 10.4647
Epoch [1/3], Step [25800/41412], Loss: 2.2943, Perplexity: 9.91797
Epoch [1/3], Step [25900/41412], Loss: 2.1913, Perplexity: 8.94735
Epoch [1/3], Step [26000/41412], Loss: 2.5850, Perplexity: 13

Epoch [2/3], Step [7700/41412], Loss: 2.1407, Perplexity: 8.50588
Epoch [2/3], Step [7800/41412], Loss: 3.2529, Perplexity: 25.8652
Epoch [2/3], Step [7900/41412], Loss: 2.2000, Perplexity: 9.02473
Epoch [2/3], Step [8000/41412], Loss: 3.0050, Perplexity: 20.1853
Epoch [2/3], Step [8100/41412], Loss: 2.0834, Perplexity: 8.03210
Epoch [2/3], Step [8200/41412], Loss: 2.0110, Perplexity: 7.47072
Epoch [2/3], Step [8300/41412], Loss: 2.1530, Perplexity: 8.61052
Epoch [2/3], Step [8400/41412], Loss: 2.3524, Perplexity: 10.5112
Epoch [2/3], Step [8500/41412], Loss: 2.5748, Perplexity: 13.1281
Epoch [2/3], Step [8600/41412], Loss: 1.6823, Perplexity: 5.37800
Epoch [2/3], Step [8700/41412], Loss: 2.0588, Perplexity: 7.83699
Epoch [2/3], Step [8800/41412], Loss: 2.4487, Perplexity: 11.5738
Epoch [2/3], Step [8900/41412], Loss: 2.3648, Perplexity: 10.6421
Epoch [2/3], Step [9000/41412], Loss: 2.0271, Perplexity: 7.59188
Epoch [2/3], Step [9100/41412], Loss: 2.1438, Perplexity: 8.53187
Epoch [2/3

Epoch [2/3], Step [32100/41412], Loss: 2.1915, Perplexity: 8.94906
Epoch [2/3], Step [32200/41412], Loss: 2.4853, Perplexity: 12.0046
Epoch [2/3], Step [32300/41412], Loss: 2.4333, Perplexity: 11.3969
Epoch [2/3], Step [32400/41412], Loss: 2.5846, Perplexity: 13.2585
Epoch [2/3], Step [32500/41412], Loss: 2.0989, Perplexity: 8.15730
Epoch [2/3], Step [32600/41412], Loss: 1.7900, Perplexity: 5.98974
Epoch [2/3], Step [32700/41412], Loss: 1.9198, Perplexity: 6.81982
Epoch [2/3], Step [32800/41412], Loss: 2.2576, Perplexity: 9.56056
Epoch [2/3], Step [32900/41412], Loss: 1.7513, Perplexity: 5.76241
Epoch [2/3], Step [33000/41412], Loss: 2.4162, Perplexity: 11.2029
Epoch [2/3], Step [33100/41412], Loss: 2.2315, Perplexity: 9.31405
Epoch [2/3], Step [33200/41412], Loss: 2.3572, Perplexity: 10.5617
Epoch [2/3], Step [33300/41412], Loss: 1.7038, Perplexity: 5.49487
Epoch [2/3], Step [33400/41412], Loss: 2.4623, Perplexity: 11.7318
Epoch [2/3], Step [33500/41412], Loss: 2.3876, Perplexity: 10.

Epoch [3/3], Step [15200/41412], Loss: 2.6300, Perplexity: 13.8738
Epoch [3/3], Step [15300/41412], Loss: 1.5398, Perplexity: 4.66367
Epoch [3/3], Step [15400/41412], Loss: 2.3941, Perplexity: 10.9581
Epoch [3/3], Step [15500/41412], Loss: 2.5298, Perplexity: 12.5516
Epoch [3/3], Step [15600/41412], Loss: 2.6900, Perplexity: 14.7316
Epoch [3/3], Step [15700/41412], Loss: 2.5771, Perplexity: 13.1586
Epoch [3/3], Step [15800/41412], Loss: 2.1969, Perplexity: 8.99680
Epoch [3/3], Step [15900/41412], Loss: 2.6481, Perplexity: 14.1271
Epoch [3/3], Step [16000/41412], Loss: 1.8789, Perplexity: 6.546471
Epoch [3/3], Step [16100/41412], Loss: 1.9358, Perplexity: 6.92957
Epoch [3/3], Step [16200/41412], Loss: 2.2462, Perplexity: 9.45139
Epoch [3/3], Step [16300/41412], Loss: 1.7797, Perplexity: 5.92811
Epoch [3/3], Step [16400/41412], Loss: 1.9333, Perplexity: 6.91237
Epoch [3/3], Step [16500/41412], Loss: 1.9836, Perplexity: 7.26888
Epoch [3/3], Step [16600/41412], Loss: 1.8847, Perplexity: 6.

Epoch [3/3], Step [39700/41412], Loss: 2.3080, Perplexity: 10.0548
Epoch [3/3], Step [39800/41412], Loss: 2.3839, Perplexity: 10.8477
Epoch [3/3], Step [39900/41412], Loss: 1.9577, Perplexity: 7.08345
Epoch [3/3], Step [40000/41412], Loss: 2.5546, Perplexity: 12.8658
Epoch [3/3], Step [40100/41412], Loss: 2.9772, Perplexity: 19.6334
Epoch [3/3], Step [40200/41412], Loss: 1.4770, Perplexity: 4.37991
Epoch [3/3], Step [40300/41412], Loss: 2.4204, Perplexity: 11.2502
Epoch [3/3], Step [40400/41412], Loss: 2.3149, Perplexity: 10.1240
Epoch [3/3], Step [40500/41412], Loss: 2.1221, Perplexity: 8.34887
Epoch [3/3], Step [40600/41412], Loss: 2.5279, Perplexity: 12.5275
Epoch [3/3], Step [40700/41412], Loss: 2.8515, Perplexity: 17.3138
Epoch [3/3], Step [40800/41412], Loss: 2.6204, Perplexity: 13.7411
Epoch [3/3], Step [40900/41412], Loss: 2.3294, Perplexity: 10.2723
Epoch [3/3], Step [41000/41412], Loss: 2.1998, Perplexity: 9.02304
Epoch [3/3], Step [41100/41412], Loss: 2.0328, Perplexity: 7.6